In [ ]:
import random
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# import warnings
# warnings.filterwarnings(action='ignore')

In [ ]:
seed = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed) # Seed 고정

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/전력사용량/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/전력사용량/test.csv')
infos = pd.read_csv('/content/drive/MyDrive/전력사용량/building_info.csv')
infos = infos.replace('-', np.NaN)
infos[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = infos[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].astype(float)
train_df.head()

In [ ]:
infos['태양광용량(kW)'] = infos['태양광용량(kW)'].fillna(0)
infos['태양광용량(kW)'] = np.where(infos['태양광용량(kW)'] > 0, 1, 0)
infos['태양광용량(kW)'].value_counts()

In [ ]:
infos.isnull().sum()

In [ ]:
train_df = pd.merge(train_df, infos[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')
test_df = pd.merge(test_df, infos[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')

In [ ]:
train_df[['풍속(m/s)', '습도(%)']] = train_df[['풍속(m/s)', '습도(%)']].fillna(train_df[['풍속(m/s)', '습도(%)']].mean())

In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:

train_df.loc[(train_df['건물번호'] == 95) & (train_df['전력소비량(kWh)'] < 1), '전력소비량(kWh)'] = np.NaN

In [ ]:
# 1
train_df.loc[770:777, '전력소비량(kWh)'] = train_df.loc[602:609, '전력소비량(kWh)'].values
train_df.loc[939:946, '전력소비량(kWh)'] = train_df.loc[602:609, '전력소비량(kWh)'].values

# 4
train_df.loc[7438:7439, '전력소비량(kWh)'] = train_df.loc[7416:7417, '전력소비량(kWh)'].values
train_df.loc[7440, '전력소비량(kWh)'] = np.nan

train_df['전력소비량(kWh)'] = train_df['전력소비량(kWh)'].interpolate(limit_direction='both', method='linear').round(3)

In [ ]:
date = pd.to_datetime(train_df.일시)
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear

date = pd.to_datetime(test_df.일시)
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear

power_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.mean).reset_index()
train_df['day_hour_mean'] = train_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['day_hour_mean'] = test_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.std).reset_index()
train_df['day_hour_std'] = train_df.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['day_hour_std'] = test_df.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.mean).reset_index()
train_df['hour_mean'] = train_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['hour_mean'] = test_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.std).reset_index()
train_df['hour_std'] = train_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['hour_std'] = test_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


power_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.mean).reset_index()
train_df['type_day_hour_mean'] = train_df.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_day_hour_mean'] = test_df.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.std).reset_index()
train_df['type_day_hour_std'] = train_df.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_day_hour_std'] = test_df.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.mean).reset_index()
train_df['type_hour_mean'] = train_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_hour_mean'] = test_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.std).reset_index()
train_df['type_hour_std'] = train_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_hour_std'] = test_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


### 공휴일 변수 추가
train_df['holiday'] = train_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train_df.loc[('20220601' <= train_df.일시)&(train_df.일시 < '20220602'), 'holiday'] = 1
train_df.loc[('20220606' <= train_df.일시)&(train_df.일시 < '20220607'), 'holiday'] = 1
train_df.loc[('20220815' <= train_df.일시)&(train_df.일시 < '20220816'), 'holiday'] = 1

test_df['holiday'] = test_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)
test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)

train_df['THI'] = 9/5*train_df['기온(C)'] - 0.55*(1-train_df['습도(%)']/100)*(9/5*train_df['습도(%)']-26)+32
test_df['THI'] = 9/5*test_df['기온(C)'] - 0.55*(1-test_df['습도(%)']/100)*(9/5*test_df['습도(%)']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train_df['CDH'] = 0
for num in range(1,100,1):
    temp = train_df[train_df['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    train_df.loc[train_df['건물번호'] == num, 'CDH'] = cdh

test_df['CDH'] = 0
for num in range(1,100,1):
    temp = test_df[test_df['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    test_df.loc[test_df['건물번호'] == num, 'CDH'] = cdh

In [ ]:
train_df['7_shifted_전력소비량'] = train_df['전력소비량(kWh)'].shift(24*7)
train_df = train_df[train_df.일시 >= '20220608'].reset_index(drop=True)

for i in train_df['건물번호'].unique():
    test_df.loc[test_df['건물번호'] == i, '7_shifted_전력소비량'] = train_df.loc[train_df['건물번호'] == i, '전력소비량(kWh)'][-7*24:].values

In [ ]:
train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형'])
train_x_type = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train_df['전력소비량(kWh)']

test_x = test_df[train_x.columns]
test_x_type = test_df[train_x_type.columns]
train_x.head()

In [ ]:
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

alpha = 25

In [ ]:
valid_df = train_x.copy()
valid_df['전력소비량(kWh)'] = 0
valid_df['pred'] = 0
pred_df = test_x.copy()
pred_df['전력소비량(kWh)'] = 0
pred_df = pred_df[['건물번호', '전력소비량(kWh)']]

labels = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for i in train_x['건물번호'].unique():
    X = train_x[train_x['건물번호'] == i].drop(columns='건물번호')
    y = train_y[train_x['건물번호'] == i]
    valid_df.loc[train_x['건물번호'] == i, '전력소비량(kWh)'] = y
    y = np.log1p(y)
    x_test = test_x[test_x['건물번호'] == i].drop(columns='건물번호')

    cols = X.columns[X.nunique() != 1]
    X = X[cols]
    x_test = x_test[cols]

    model = XGBRegressor(random_state=seed)

    results = []

    for fold_, (train_index, val_index) in enumerate(skf.split(X, X['day'])):

        x_train, x_valid = X.iloc[train_index], X.iloc[val_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[val_index]

        model.set_params(**{'objective':weighted_mse(alpha)})
        model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=0)

        val_pred = model.predict(x_valid)
        valid_df.loc[x_valid.index, 'pred'] = np.expm1(val_pred)
        pred = model.predict(x_test)
        results.append(pred)

    preds = np.mean(results, axis=0)
    preds = np.expm1(preds)
    pred_df.loc[test_x['건물번호'] == i, '전력소비량(kWh)'] = preds

    labels.append([valid_df.loc[train_x['건물번호'] == i, '전력소비량(kWh)'], preds])

val_score = SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])

In [ ]:
train_x_type["건물번호"] = train_x_type["건물번호"].astype("category")
test_x_type["건물번호"] = test_x_type["건물번호"].astype("category")

valid_df_type = train_x_type.copy()
valid_df_type['전력소비량(kWh)'] = 0
valid_df_type['pred'] = 0
pred_df_type = test_x_type.copy()
pred_df_type['전력소비량(kWh)'] = 0
pred_df_type = pred_df_type[['건물유형', '전력소비량(kWh)']]

labels_type = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for i in train_x_type['건물유형'].unique():
    X = train_x_type[train_x_type['건물유형'] == i].drop(columns='건물유형')
    y = train_y[train_x_type['건물유형'] == i]
    valid_df_type.loc[train_x_type['건물유형'] == i, '전력소비량(kWh)'] = y
    y = np.log1p(y)
    x_test = test_x_type[test_x_type['건물유형'] == i].drop(columns='건물유형')

    model = XGBRegressor(random_state=seed, tree_method="hist", enable_categorical=True)

    results = []

    for fold_, (train_index, val_index) in enumerate(skf.split(X, X['day'])):

        x_train, x_valid = X.iloc[train_index], X.iloc[val_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[val_index]

        model.set_params(**{'objective':weighted_mse(alpha)})
        model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=0)

        val_pred = model.predict(x_valid)
        valid_df_type.loc[x_valid.index, 'pred'] = np.expm1(val_pred)
        pred = model.predict(x_test)
        results.append(pred)


    preds = np.mean(results, axis=0)
    preds = np.expm1(preds)
    pred_df_type.loc[test_x_type['건물유형'] == i, '전력소비량(kWh)'] = preds

    labels_type.append([valid_df_type.loc[train_x_type['건물유형'] == i, '전력소비량(kWh)'], preds])

type_val_score = SMAPE(valid_df_type['전력소비량(kWh)'], valid_df_type['pred'])

In [ ]:
val_score, type_val_score, np.mean([val_score, type_val_score])

In [ ]:
from sklearn.linear_model import LinearRegression

counts = 0
w = 0.98

for num in train_df['건물번호'].unique():
    X = pd.concat([valid_df.loc[valid_df['건물번호'] == num, 'pred'], valid_df_type.loc[valid_df_type['건물번호'] == num, 'pred']], axis=1).values
    y = valid_df.loc[valid_df['건물번호'] == num, '전력소비량(kWh)']
    reg = LinearRegression().fit(X, y)
    p = reg.predict(X)
    score = SMAPE(valid_df.loc[valid_df['건물번호'] == num, '전력소비량(kWh)'], p)
    pred_x = pd.concat([pred_df.loc[pred_df['건물번호'] == num, '전력소비량(kWh)'], pred_df_type.loc[pred_df['건물번호'] == num, '전력소비량(kWh)']], axis=1).values
    pred = reg.predict(pred_x)
    threshold = train_df.loc[train_df['건물번호'] == num, '전력소비량(kWh)'].iloc[-24*28:].min()
    counts += sum(pred < threshold*w)
    pred_df.loc[pred_df['건물번호'] == num, '전력소비량(kWh)'] = np.where(pred < threshold*w, threshold*w, pred)
    valid_df.loc[valid_df['건물번호'] == num, 'pred'] = p

SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])

In [ ]:
# plt.figure(figsize=(26, 300))
# for i in train_df['건물번호'].unique():
#     plt.subplot(train_df['건물번호'].nunique(), 1, i)
#     plt.title(i)
#     plt.plot(range(len(labels[i-1][0])), labels[i-1][0])
#     plt.plot(range(len(labels[i-1][0]), len(labels[i-1][0])+len(labels[i-1][1])), pred_df[pred_df['건물번호']==i]['전력소비량(kWh)'])
# plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

submission = pd.read_csv('/content/drive/MyDrive/전력사용량/sample_submission.csv')
submission['answer'] = pred_df['전력소비량(kWh)']
submission.to_csv('./submit.csv', index=False)
submission